In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import re

import numpy as np

from markov_model import *

import pickle

from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
markov_order = 1

In [68]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/motif_predictions/split_75_25/'

In [69]:
motifs = pd.read_csv(data_dir + '../motifs.csv').motif.unique()

In [70]:
#see select_data.ipynb

train_df = pd.read_csv(data_dir + 'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')

In [71]:
# get the frequency counts of all motifs till 11mer
kmer_train = KmerCount(2,pseudocount=0.1)

kmer_train.compute_counts(train_df.seq)

100%|██████████| 13600/13600 [00:28<00:00, 480.98it/s]


In [72]:
markov_model = MarkovChain(kmer_train)

In [73]:
# calculate the markov matrix using the 11mer counts
markov_model.compile_from_counts()

In [75]:
test_df = test_df.set_index('seq_name').seq #dataframe to series

In [76]:
#inference

mapping = {'A':0,'C':1,'G':2,'T':3}

motif_probas = []

for seq_name, seq in tqdm(test_df.items(),total = len(test_df)):
    
    prbs = markov_model.impute_for_seq(seq, order=markov_order)
    
    seq_mapping = [mapping[base] for base in seq] #motif sequence to ordinal encoding
    
    target_probas = [prob[seq_mapping[idx]] for idx, prob in enumerate(prbs)] #take the right base
    
    motif_probas.append((seq_name, target_probas))
        
    #for motif in motifs:
    #    
    #    motif_mapping = [mapping[base] for base in motif] #motif sequence to ordinal encoding
    #
    #    for match in re.finditer(motif, seq):
    #
    #        target_probas = prbs[match.start():match.end()] #probabilities for ACGT at each motif position
    #
    #        target_probas = [prob[motif_mapping[idx]] for idx, prob in enumerate(target_probas)] #take the right base 
    #
    #        avg_target_probas = np.mean(target_probas)
    #        
    #        motif_probas.append((seq_name, motif,match.start(),avg_target_probas))

100%|██████████| 4534/4534 [00:23<00:00, 194.55it/s]


In [36]:
#motif_probas = pd.DataFrame(motif_probas, columns=['seq_name','motif','motif_start','avg_target_prob'])

#motif_probas.to_csv(data_dir + '11_mer.tsv', sep='\t', index=None)

In [77]:
with open(data_dir + 'dinucl.pickle', 'wb') as f:
    pickle.dump(motif_probas,f)